In [1]:
import xlearn as xl
import os
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import joblib
import ast
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
import numpy as np
from collections import defaultdict
import random

# 모델 학습

In [2]:
user_libffm_path = '../Dataset/user_libffm_data_folder/'
lib_path = '../Dataset/lib_dataset/'
user_libffm_result_path = '../Dataset/user_FFM_result_folder/'
encoder_data_path = '../DataSet/Encoder/'
model_path = 'ffm_model/'

In [3]:
#reg

ffm_reg_model = xl.create_ffm()
ffm_reg_model.setTrain(os.path.join(lib_path, "FFM_reg.txt"))

param = {'task':'reg', 'lr':0.02, 'lambda':0.002, 'opt':'ftrl', 'metric': 'rmse', 'epoch':100}

ffm_reg_model.fit(param, os.path.join(model_path, "ffm_reg_model.out"))

ffm_reg_model.setTest(os.path.join(lib_path, "FFM_test_reg.txt"))
ffm_reg_model.predict(os.path.join(model_path, "ffm_reg_model.out"), os.path.join(lib_path, "reg_output.txt"))

In [4]:

y_reg_true = []
with open(os.path.join(lib_path, "FFM_valid_reg.txt"), 'r') as file:
    for line in file:
        first_value = int(line.split()[0])
        y_reg_true.append(first_value)

y_reg_scores = []
with open(os.path.join(lib_path, "reg_output.txt"), 'r') as file:
    for line in file:
        value = float(line.strip())
        y_reg_scores.append(value)


rmse_score = np.sqrt(mean_squared_error(y_reg_true, y_reg_scores)) 
print(rmse_score)

2.5507344379718315


In [5]:
#binary

ffm_binary_model = xl.create_ffm()
ffm_binary_model.setTrain(os.path.join(lib_path, "FFM_binary.txt"))

param = {'task':'binary', 'lr':0.02, 'lambda':0.002, 'opt':'ftrl', 'metric': 'auc','epoch':100}
ffm_binary_model.fit(param, os.path.join(model_path, "ffm_binary_model.out"))

ffm_binary_model.setSigmoid()
ffm_binary_model.setTest(os.path.join(lib_path, "FFM_test_binary.txt"))
ffm_binary_model.predict(os.path.join(model_path, "ffm_binary_model.out"), os.path.join(lib_path, "binary_output.txt"))

In [6]:
y_binary_true = []
with open(os.path.join(lib_path, "FFM_valid_binary.txt"), 'r') as file:
    for line in file:
        first_value = int(line.split()[0])
        y_binary_true.append(first_value)

y_binary_scores = []
with open(os.path.join(lib_path, "binary_output.txt"), 'r') as file:
    for line in file:
        value = float(line.strip())
        y_binary_scores.append(value)


auc_score = np.sqrt(roc_auc_score(y_binary_true, y_binary_scores))
print(auc_score)

0.8902246678891403


# 문제 추천 메소드

In [7]:

def get_binary_recommend_result(user_name):

    problem_number_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_number_Encoder.joblib'))
    problem_algorithm_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_algorithm_Encoder.joblib'))
    
    ffm_binary_model = xl.create_ffm()
    ffm_binary_model.setSigmoid()
    ffm_binary_model.setTest(os.path.join(user_libffm_path, f"{user_name}_libffm_binary.txt"))
    ffm_binary_model.predict(os.path.join(model_path, "ffm_binary_model.out"), os.path.join(user_libffm_result_path, f"{user_name}_ffm_binary_result.txt"))

    result = []
    with open(os.path.join(user_libffm_result_path, f"{user_name}_ffm_binary_result.txt")) as file:
        for line in file:
            value = float(line.strip())
            result.append(value)

    recommend_dict = defaultdict(list)

    with open(os.path.join(user_libffm_path, f"{user_name}_libffm_binary.txt"), 'r') as file:
        user_tier = re.findall(r'1:(\d+):[.\d]+', file.readline())
        user_weak_algorithm = re.findall(r'6:(\d+):[.\d]+', file.readline())

        file.seek(0)
        
        for i, line in enumerate(file):
            values = (re.findall(r'2:(\d+):1', line), re.findall(r'3:(\d+):0.5', line))
            recommend_dict[int(values[1][0])].append(((problem_number_Encoder.inverse_transform([int(values[0][0])]))[0] , result[i]))

    sorted_dict = {key: sorted(value, key=lambda x: -x[1]) for key, value in recommend_dict.items()}
    user_weak_algorithm = [int(x) for x in user_weak_algorithm]
    user_weak_algorithm = problem_algorithm_Encoder.inverse_transform(user_weak_algorithm)

    
    
    return sorted_dict, user_tier, user_weak_algorithm

In [8]:
def get_reg_recommend_result(user_name):

    problem_number_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_number_Encoder.joblib'))
    problem_algorithm_Encoder = joblib.load(os.path.join(encoder_data_path, 'problem_algorithm_Encoder.joblib'))
    
    ffm_reg_model = xl.create_ffm()
    ffm_reg_model.setTest(os.path.join(user_libffm_path, f"{user_name}_libffm_reg.txt"))
    ffm_reg_model.predict(os.path.join(model_path, "ffm_reg_model.out"), os.path.join(user_libffm_result_path, f"{user_name}_ffm_reg_result.txt"))

    result = []
    with open(os.path.join(user_libffm_result_path, f"{user_name}_ffm_reg_result.txt")) as file:
        for line in file:
            value = float(line.strip())
            result.append(value)

    recommend_dict = defaultdict(list)

    with open(os.path.join(user_libffm_path, f"{user_name}_libffm_reg.txt"), 'r') as file:
        user_tier = re.findall(r'1:(\d+):[.\d]+', file.readline())
        user_weak_algorithm = re.findall(r'6:(\d+):[.\d]+', file.readline())

        file.seek(0)
        
        for i, line in enumerate(file):
            values = (re.findall(r'2:(\d+):1', line), re.findall(r'3:(\d+):0.5', line))
            recommend_dict[int(values[1][0])].append(((problem_number_Encoder.inverse_transform([int(values[0][0])]))[0] , result[i]))
    
    sorted_dict = {key: sorted(value, key=lambda x: -x[1]) for key, value in recommend_dict.items()}
    user_weak_algorithm = [int(x) for x in user_weak_algorithm]
    user_weak_algorithm = problem_algorithm_Encoder.inverse_transform(user_weak_algorithm)
    
    return sorted_dict, user_tier, user_weak_algorithm

# 모델 출력 기반 Rule Base

In [9]:

def get_recommend_list(user_name):
    
    binary_dict, user_tier, user_weak_algorithm = get_binary_recommend_result(user_name)
    reg_dict, user_tier, user_weak_algorithm = get_reg_recommend_result(user_name)

    user_tier = int(user_tier[0])
    user_weak_algorithm = list(user_weak_algorithm)

    fin_binary_keys = [x for x in range(6, user_tier + 4) if x<=22]
    fin_binary_dict = {key: binary_dict[key] for key in fin_binary_keys}

    fin_reg_keys = [x for x in range(6, user_tier + 4) if x<=22]
    fin_reg_dict = {key: reg_dict[key] for key in fin_reg_keys}

    fin_binary_dict = {key: [x[0] for x in value[:10]] for key, value in fin_binary_dict.items()}
    fin_reg_dict = {key: [x[0] for x in value[:10]] for key, value in fin_reg_dict.items()}

    fin_dict = {key: list(set(fin_reg_dict[key]+fin_binary_dict[key])) for key in fin_reg_dict.keys()}
    
    selected_keys = random.sample(fin_dict.keys(), 3)
    selected_values = [random.choice(fin_dict[key]) for key in selected_keys]


    return selected_values, user_weak_algorithm
    

In [10]:
get_recommend_list('llsy159')

C:\Users\win10KDH\AppData\Local\Temp\ipykernel_59096\428334463.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  selected_keys = random.sample(fin_dict.keys(), 3)


([26259, 27436, 23973], ['다이나믹 프로그래밍', '그래프 이론', '구현', '그래프 탐색', '자료 구조'])

# Optuna

In [ ]:
!pip install optuna

     -------------------------------------- 409.6/409.6 kB 8.5 MB/s eta 0:00:00
     -------------------------------------- 226.8/226.8 kB 7.0 MB/s eta 0:00:00
     ---------------------------------------- 78.7/78.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error

ffm_model = xl.create_ffm()
ffm_model.setTrain("FFM_train_reg.txt")
ffm_model.setValidate("FFM_test_reg.txt")

y_true = []
with open('FFM_test_reg.txt', 'r') as file:
    for line in file:
        first_value = int(line.split()[0])
        y_true.append(first_value)

def objective_function(trial):

    param = {
        #'task':'binary',
        'task':'reg',
        'lr':trial.suggest_categorical('lr', [0.2,0.02,0.002, 0.0002,0.5]),
        'lambda':trial.suggest_categorical('lambda', [0.002, 0.0002, 0.00002,0.000002]),
        'opt':'ftrl', 
        'metric': 'rmse',
        'alpha':trial.suggest_categorical('alpha', [0.002,0.02,0.0002]),
        'beta':trial.suggest_categorical('beta', [0.8, 0.08, 0.2]),
        'lambda_1':trial.suggest_categorical('lambda_1', [0.001,0.01,0.0001,0.005]),
        'lambda_2': trial.suggest_categorical('lambda_2', [1.0,1.5,0.5,0.1]),
        'k':trial.suggest_categorical('k', [2,4,5,8]),
        'init':trial.suggest_categorical('init', [0.1,0.4,0.8,0.66])
    }

    ffm_model.fit(param, "ffm_model.out")
    ffm_model.setTest("FFM_test_reg.txt")
    ffm_model.predict("ffm_model.out", "output.txt")

    y_scores = []

    with open('output.txt', 'r') as file:
        for line in file:
            value = float(line.strip())
            y_scores.append(value)

    result = np.sqrt(mean_squared_error(y_true, y_scores))
    print(result)

    return result


study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(lambda trial : objective_function(trial), n_trials=1000)
print('Best trial : score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2023-11-14 13:51:07,410] A new study created in memory with name: no-name-16547e71-f89d-4aa6-acf2-128e14b0ea9e
[I 2023-11-14 13:51:32,182] Trial 0 finished with value: 3.0700969901270496 and parameters: {'lr': 0.0002, 'lambda': 0.002, 'alpha': 0.002, 'beta': 0.2, 'lambda_1': 0.01, 'lambda_2': 1.0, 'k': 8, 'init': 0.66}. Best is trial 0 with value: 3.0700969901270496.


3.0700969901270496


[I 2023-11-14 13:51:53,082] Trial 1 finished with value: 3.0196010716890256 and parameters: {'lr': 0.02, 'lambda': 0.0002, 'alpha': 0.02, 'beta': 0.08, 'lambda_1': 0.01, 'lambda_2': 1.0, 'k': 5, 'init': 0.8}. Best is trial 1 with value: 3.0196010716890256.


3.0196010716890256


[I 2023-11-14 13:52:17,221] Trial 2 finished with value: 3.062891097158365 and parameters: {'lr': 0.0002, 'lambda': 0.002, 'alpha': 0.002, 'beta': 0.8, 'lambda_1': 0.01, 'lambda_2': 0.5, 'k': 8, 'init': 0.1}. Best is trial 1 with value: 3.0196010716890256.


...

2.9195274397756847


[I 2023-11-14 14:02:04,589] Trial 34 finished with value: 2.9195240472807766 and parameters: {'lr': 0.002, 'lambda': 2e-05, 'alpha': 0.02, 'beta': 0.08, 'lambda_1': 0.001, 'lambda_2': 0.1, 'k': 5, 'init': 0.4}. Best is trial 34 with value: 2.9195240472807766.


2.9195240472807766


[I 2023-11-14 14:02:27,631] Trial 35 finished with value: 2.9192835012658396 and parameters: {'lr': 0.02, 'lambda': 0.0002, 'alpha': 0.02, 'beta': 0.08, 'lambda_1': 0.001, 'lambda_2': 0.1, 'k': 5, 'init': 0.1}. Best is trial 35 with value: 2.9192835012658396.


2.9192835012658396
